# Views
- Vistas basadas en funciones
- Vistas basadas en clases

## Para crear una aplicacion en Django
- python manage.py startapp
- python manage.py startapp ecommerce

## Para entrar en el container


In [ ]:
- docker exec -it hellodjango-web-1 bash
- muestra como entro: python@6533204c6cff:/app/src$ 
 - If not running
 - docker compose --profile web up -d web

## CRUD
Create, Retrieve, Update, Delete

## Views.py

In [ ]:
from django.shortcuts import render
from django.http import HttpRequest, HttpResponse

# Create your views here.
def home(request: HttpRequest) -> HttpResponse:
    return HttpResponse("Welcome to the E-commerce Home Page")

## Urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.home, name="home"),
]


## Config/urls.py

In [ ]:
urlpatterns = [
    path("up/", include("up.urls")),
    path("", include("pages.urls")),
    path("ecommerce/", include("ecommerce.urls")), #<-- Added line to include ecommerce URLs
    path("admin/", admin.site.urls),
]

Urls for predefined DetailViews
products/urls.py

from products.views import ProductListView, ProductDetailView
urlpatterns = [
    path("products/", ProductListView.as_view()),
    path("products/<int:pk>", ProductDetailView.as_view())
]

## ecommerce/urls.py
Redirect

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.home, name="home"),
    path("redirect/", views.redirect_to_home, name="redirect_to_home"),
]


### Models.py

In [ ]:
from django.db import models

# Create your models here.
class ProductModel(models.Model):
    title = models.TextField()
    price = models.FloatField()

## Correr migraciones
- Por cada cambio como crear o borrar un campo del modelo se debe correr


In [ ]:
python manage.py makemigrations
python manage.py migrate

In [ ]:
python@6533204c6cff:/app/src$ python manage.py makemigrations
Migrations for 'ecommerce':
  ecommerce/migrations/0001_initial.py
    + Create model ProductModel

- Migrate

In [ ]:
Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... OK
  Applying auth.0012_alter_user_first_name_max_length... OK
  Applying ecommerce.0001_initial... OK
  Applying sessions.0001_initial... OK

## Borrar Migraciones y Compresion
- Se debe comprimir migraciones antes de subir al repo
- python manage.py squashmigrations <APP_LABEL> <MIGRATION_NUMBER>
- Example: python manage.py squashmigrations ecommerce 0004
- python manage.py migrate

## Registrar modelos en Admin

In [ ]:
from django.contrib import admin

# Register your models here.
from .models import ProductModel

admin.site.register(ProductModel)

## Create user type Admin

In [ ]:
python manage.py createsuperuser

- Password must be at least 8 charachters
 It must contain at least 8 characters.

 - Endpoint admin
 http://0.0.0.0:8000/admin/login/?next=/admin/


## Tipos basicos de Vistas
Dentro de Users en admin
- Listas View donde ves los usuarios creados.
- Create View donse puedes insertar los datos para crear un usuario (add user)
- Retrive and Update View al dar click en un usuario creado, para modificarlo
- Delete View para elmininar

## Templates
Uso de templates en Views

In [ ]:
def product_model_list_view(request: HttpRequest) -> HttpResponse:
    products = ProductModel.objects.all()
    template = "ecommerce/product_list.html"
    context = {"products": products}
    return render(request, template, context)

Create the template:
 - src/pages/templates/ecommerce/product_list.html

In [ ]:
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Lista de Productos</title>
  </head>
  <body>
    {% for product in products %}
    <ul>
      <li>{{ product.title }} - ${{ product.price }}</li>
    </ul>
    {% endfor %}
  </body>
</html>


## Protegiendo los endpoints

- Add on config/settings.py the url as
- LOGIN_REDIRECT_URL = "/admin/login/"

and on views add the decorator as:

In [ ]:
from django.contrib.auth.decorators import login_required

# Create your views here.
@login_required(login_url='/admin/login/')
def product_model_list_view(request: HttpRequest) -> HttpResponse:
    products = ProductModel.objects.all()
    template = "ecommerce/product_list.html"
    context = {"products": products}
    
    if request.user.is_authenticated:
        template = "ecommerce/product_list.html"
    else:
        template = "ecommerce/product_list_public.html"
    
    return render(request, template, context)

## Vista de Detalle

In [ ]:
def product_model_detail_view(request: HttpRequest, product_id: int) -> HttpResponse:
    instance = get_object_or_404(ProductModel, id=product_id)
    template = "ecommerce/product_detail.html"
    context = {"product": instance}
    
    return render(request, template, context)

Using models

import from django.views.generic import DetailView
from .models import Product

class ProductDetailView(DetailView):
  model = Product


## Vista de Creacion

In [ ]:
def product_model_create_view(request: HttpRequest) -> HttpResponse:
    form = ProductModelForm(request.POST or None)
    if form.is_valid():
        instance = form.save(commit=False)
        instance.save()
        messages.success(request, "Produto creado con exito.")
        return HttpResponseRedirect("/ecommerce/{product_id}/".format(product_id=instance.id))
    template = "ecommerce/product_create.html"
    context = {"form": form}
    return render(request, template, context)

## Vista de Actualizacion

In [ ]:
def product_model_update_view(request: HttpRequest, product_id: int) -> HttpResponse:
    instance = get_object_or_404(ProductModel, id=product_id)
    form = ProductModelForm(request.POST or None, instance=instance)
    if form.is_valid():
        instance = form.save(commit=False)
        instance.save()
        messages.success(request, "Produto actualizado con exito.")
        return HttpResponseRedirect("/ecommerce/{product_id}/".format(product_id=instance.id))
    template = "ecommerce/product_update.html"
    context = {"form": form}
    return render(request, template, context)

## Vista de Eliminar

In [ ]:
def product_model_delete_view(request: HttpRequest, product_id: int) -> HttpResponse:
    instance = get_object_or_404(ProductModel, id=product_id)
    if request.method == "POST":
        instance.delete()
        messages.success(request, "Produto eliminado con exito.")
        return HttpResponseRedirect("/ecommerce/")
    template = "ecommerce/product_delete.html"
    context = {"product": instance}
    
    return render(request, template, context)

## Busqueda en Vista de Listado

- Crea template ecommerce/product_search.html
- Update view.py

In [ ]:
@login_required(login_url='/admin/login/')
def product_model_list_view(request: HttpRequest) -> HttpResponse:
    query = request.GET.get("q", None)
    queryset = ProductModel.objects.all()
    if query is not None:
        queryset = queryset.filter(
            Q(title__icontains=query) |
            Q(description__icontains=query)
        )
    products = queryset
    template = "ecommerce/product_list.html"
    context = {"products": products}
    
    if request.user.is_authenticated:
        template = "ecommerce/product_list.html"
    else:
        template = "ecommerce/product_list_public.html"
    
    return render(request, template, context)

## Guardar data usando el shell de Django
- python manage.py shell
- dentro del shell importo el modelo
 - from ecommerce.models import ProductModel
 - creamos un objeto nuevo: ProductModel.objects.create(title="laptop2", price=2000)

Tambien puedes hacer queries.
- Queryset
 - queryset = ProductModel.objects.all() o qs = ProductModel.objects.all()

- Filtrar
 - queryset.filter(title__icontains="producto")
   - Answer: <Queryset [ProductModel: ProductModel object (17)]>
 - my_product = ProductModel.objects.get(id=17)
 - my_product.title 
    - Answer: "Product 1"
- Editar el producto
  - my_product.price = 249
  - my_product.save()
  - exit()

## Validation

In [ ]:
from django.core.exceptions import ValidationError

BLOCKER_WORDS = ['barato', 'malo']

def validate_for_blocker_words(value: str) -> str:
    init_string = f"{value}".lower()
    unique_words = set(init_string.split())
    blocked_words = set(BLOCKER_WORDS)
    invalid_words = (unique_words & blocked_words)
    has_error = len(invalid_words) > 0
    if has_error:
        errors: list[str] = []
        for word in invalid_words:
            msg = "{} es una palabra bloqueada.".format(word)
            errors.append(msg)
        raise ValidationError(errors) # type: ignore
    return value

Update the model to do the validation

In [ ]:
    def save(self, *args: Any, **kwargs: Any) -> None:
        validate_for_blocker_words(self.title)
        super().save(*args, **kwargs)

Agregar opciones a la validacion

In [ ]:
class ProductModel(models.Model):

 class ProductStateOptions(models.TextChoices):
        PUBLISHED = "PB", "Published" #<---Added
        DRAFT = "DR", "Draft"
        BROKEN = "BR", "Broken"

    state = models.CharField(max_length=2,  choices=PUBLISH_STATE_CHOICES, default=ProductStateOptions.DRAFT) #<---Added
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(default="No description provided.")
    seller = models.CharField(max_length=100, default="Unknown Seller")
    color = models.CharField(max_length=50, default="black")
    product_dimensions = models.CharField(max_length=20, default="Not specified")

    def save(self, *args: Any, **kwargs: Any) -> None:
        validate_for_blocker_words(self.title)
        super().save(*args, **kwargs)

    def is_published(self) -> bool:
        return self.state == "PB" #<---Added

## Agregar Modelo Abstracto como base


In [ ]:
python manage.py startapp base

El comando crea un folder as src/base, este sirve para crear y compartir comportamientos comunes
- en src/base/models.py
- se utiliza como esqueleto para otras funciones en ecommerce

In [ ]:
from django.db import models
from django.utils import timezone

# Create your models here.
from typing import Any
from django.db import models

# Create your models here.
class BasePublishModel(models.Model):
    class ProductStateOptions(models.TextChoices):
        PUBLISHED = "PB", "Published"
        DRAFT = "DR", "Draft"
        BROKEN = "BR", "Broken"

    state = models.CharField(max_length=2,  choices=ProductStateOptions.choices, default=ProductStateOptions.DRAFT)
    timestamp = models.DateTimeField(auto_now_add=True)
    updated = models.DateTimeField(auto_now=True)
    publish_timestamp = models.DateTimeField(auto_now_add=False, auto_now=False, null=True)

    class Meta:
        abstract = True
        ordering = ['-publish_timestamp', '-updated', '-timestamp']
    def save(self, *args: Any, **kwargs: Any) -> None:
        if self.state_is_published and self.publish_timestamp is None:
            self.publish_timestamp = timezone.now()
        else:
            self.publish_timestamp = None
        super().save(*args, **kwargs)

    @property
    def state_is_published(self) -> bool:
        return self.state == self.ProductStateOptions.PUBLISHED.value

    def is_published(self) -> bool:
        publish_timestamp = self.publish_timestamp or self.timestamp
        return self.state_is_published and publish_timestamp < timezone.now()

instanciamos en ecommerce/models.py

In [ ]:
from base.models import BasePublishModel

# Create your models here.
class ProductModel(BasePublishModel):
    
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(default="No description provided.")
    seller = models.CharField(max_length=100, default="Unknown Seller")
    color = models.CharField(max_length=50, default="black")
    product_dimensions = models.CharField(max_length=20, default="Not specified")

    def save(self, *args: Any, **kwargs: Any) -> None:
        validate_for_blocker_words(self.title)
        super().save(*args, **kwargs)

    def is_published(self) -> bool:
        return self.state == "PB"

agregamos el base model en config/settings

In [ ]:
INSTALLED_APPS = [
    "pages.apps.PagesConfig",
    "ecommerce.apps.EcommerceConfig", 
    "base.apps.BaseConfig",#<--- Added base app
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]

## Creacion a granel (varios grupos al mismo tiempo)
Usar el shell para hacerlo

In [ ]:
products_data = [
{"title": "Producto1", "Price": "345"},
{"title": "Producto2", "Price": "245"}, 
{"title": "Producto3", "Price": "335"}, 
{"title": "Producto4", "Price": "445"}]

for i in range(10):
    new_data = ("title": "Producto {}".format(i), "price": i*100+99.99)
    products_data.append(new_data)

# Pasando al Modelo

from ecommerce.models import ProductModel

new_objects = []
for product_data in products_data:
    print(product_data)
    new_objects.append(ProductModel(**product_data))

# Guardandolo
ProductModel.objects.bulk_create(new_objects, ignore_conflicts=True)

## SlugFields y Senales en Modelos
Agregar slugs a las urls del producto

-Senales
    - pre save
    - post save
    - pre delete
    - post delete
    - pre init
    - post init
    - pre_migrate
    - post_migrate

In [ ]:
En el shell para ver todas las signals:
 from django.db.models import signals
 dir(signals)

Va a regresar:
['ModelSignal, Signal, __builtins__, __cached__, __doc__, __file__, __loader__, __name__, post_migrate, .... ]

Actualizar modelo

In [ ]:
from django.db.models.signals import pre_save #<---Added
from django.utils.text import slugify #<---Added

# Create your models here.
class ProductModel(BasePublishModel):
    
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(default="No description provided.")
    seller = models.CharField(max_length=100, default="Unknown Seller")
    color = models.CharField(max_length=50, default="black")
    product_dimensions = models.CharField(max_length=20, default="Not specified")
    slug = models.SlugField(unique=True, blank=True, db_index=True) #<---Added

    def get_absolute_url(self) -> str:
        return f"/products/{self.slug}/"
    
    def save(self, *args: Any, **kwargs: Any) -> None:
        validate_for_blocker_words(self.title)
        super().save(*args, **kwargs)

    def is_published(self) -> bool:
        return self.state == "PB"
    
def slugify_pre_save(sender: type[ProductModel], instance: ProductModel, *args: Any, **kwargs: Any) -> None:  #<---Added
    if instance.slug == "":
        new_slug = slugify(instance.title)
        MyModel = instance.__class__
        qs = MyModel.objects.filter(slug__startswith=new_slug).exclude(pk=instance.pk)
        if qs.count() == 0:
            instance.slug = new_slug
        else:
            instance.slug = f"{new_slug}-{qs.count() + 1}"
pre_save.connect(slugify_pre_save, sender=ProductModel) #<---Added

## Fixtures para Cargar Data
Salvar la data en archivo json localizado en ecommerce/fixtures para pruebas
- Crea file ecommerce/fixtures/ProductModel.json

In [ ]:
python manage.py dumpdata ecommerce --indent 4 --format json > ecommerce/fixtures/ProductModel.json

- Convertir de json a db

In [ ]:
python manage.py loaddata ecommerce/fixtures/ProductModel.json

## Llaves Foraneas
Ejemplo relacionar productos con usuarios

In [ ]:
from django.conf import settings

User = settings.AUTH_USER_MODEL
# Create your models here.
class ProductModel(BasePublishModel):
    
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(default="No description provided.")
    seller = models.CharField(max_length=100, default="Unknown Seller")
    color = models.CharField(max_length=50, default="black")
    product_dimensions = models.CharField(max_length=20, default="Not specified")
    slug = models.SlugField(unique=True, blank=True, db_index=True)
    user = models.ForeignKey(User, null=True, on_delete=models.CASCADE, related_name="products") #CASCADE deletes all the products of the user, SET_NULL does not

## Vistas basadas en clases

In [ ]:
class ProductListView(ListView):
    queryset = Product.objects.all()

product_list_view = require_http_methods(["GET"])(ProductListView.as_view())


## TemplateView
Templates genericas como about.html

In [ ]:
from django.views.generic import  TemplateView


class AboutView(TemplateView):
    template_name = "about.html"

    OR

class AboutView(View):
    def get(self, request):
        return render(request, "about.html", {})


Update the urls

In [ ]:
from django.urls import path
from django.views.generic import TemplateView
from src.base import admin

urlpatterns = [
    path("admin/", admin.site.urls), 
    path("about/", TemplateView.as_view(template_name="about.html")), #<-- Added TemplateView URL
]

## Redirect View

In [ ]:
def about_us_redirect_view(request):
    return HttpResponseRedirect("/about/")

class AboutUsRedirectView(RedirectView):
    url = "/about/"

urlpatterns = [
    path("admin/", admin.site.urls), 
    path("about/", TemplateView.as_view(template_name="about.html")),
    path("about-us/", RedirectView.as_view(url="/about/")), #<-- Added delete view URL
]

## Obtener data de contexto

In [ ]:
class ProductListView(ListView):
    model = Product

    def get_context_data(self, **args, **kwargs: Any) -> dict[str, Any]:
        return super().get_context_data(**args,**kwargs)

## Proxy

En models.py

In [ ]:
class DigitalProduct(Product):
   class Meta:
         proxy = True

En Urls

In [ ]:
urlpatterns = [
    path("admin/", admin.site.urls), 
    path("about/", TemplateView.as_view(template_name="about.html")),
    path("about-us/", RedirectView.as_view(url="/about/")),
    path("products/", ProductListView.as_view()), #<-- Added delete view URL
    path("digital-products/", DigitalProductListView.as_view()), #<-- Added delete view URL
]

En Views

In [ ]:
class DigitalProductListView(ListView):
    model = DigitalProduct

    def get_context_data(self, **args, **kwargs: Any) -> dict[str, Any]:
        return super().get_context_data(**args,**kwargs)

En Admin

In [ ]:
from django.contrib import admin

# Register your models here.
from .models import DigitalProduct, Product
admin.site.register(Product)
admin.site.register(DigitalProduct)

## Mixin

products/mixins.py

In [ ]:
class TemplateTitleMixin(object):
    title = None
    def get_context_data(self, **args, **kwargs) -> dict[str, str]:
        context = super().get_context_data(**args, **kwargs)
        context["title"] = self.get_title()
        return context
    
    def get_title(self) -> str:
        if self.title is None:
            return "Default Title"
        return self.title

include in views

In [ ]:
class ProductListView(TemplateTitleMixin, ListView):
    model = Product
    template = "products/product_list.html"
    title = "Product List"

    # With mixin, no need context data
    # def get_context_data(self, **args, **kwargs: Any) -> dict[str, Any]:
    #     return super().get_context_data(**args,**kwargs)

en template

In [ ]:
 <body>
    {% if title %}
    <h1>{{ title }}</h1>
    {% end if %}

### Mixin para proteger por login

In [ ]:
from django.contrib.auth.mixins import LoginRequiredMixin

class ProtectedProductDetailView(LoginRequiredMixin, DetailView):
    model = Product

Agregar en urls

In [ ]:
urlpatterns = [
    path("admin/", admin.site.urls), 
    path("about/", TemplateView.as_view(template_name="about.html")),
    path("about-us/", RedirectView.as_view(url="/about/")),
    path("products/", ProductListView.as_view()), 
    path("digital-products/", DigitalProductListView.as_view()), 
    path("my-products/<slug:slug>/", ProtectedProductDetailView.as_view()), #<-- Added delete view URL
]

## Redirects View basadas en instancia de modelo

src/products/views/py

In [ ]:
class ProductRedirectView(RedirectView):

    def get_redirect_url(self, *args, **kwargs):
        url_params = self.kwargs
        pk = url_params.get("pk")
        obj = get_object_or_404(Product, pk=pk)
        slug = obj.pk
        return f"/products/products/{slug}/"

## Model Forms

products/models.py

In [ ]:
from django.conf import settings

User = settings.AUTH_USER_MODEL

# Create your models here.
class Product(models.Model):
    user = models.ForeignKey(User, blank=True, null=True, on_delete=models.SET_NULL)
    title = models.CharField(max_length=255)
    price = models.DecimalField(max_digits=10, decimal_places=2)

## Create Protected Views

In [ ]:
class ProtectedProductCreateView(LoginRequiredMixin, CreateView):
    form_class = ProductModelForm
    model = Product
    template_name = "products/product_form.html"
    success_url = "/products/"

    def form_valid(self, form):
        form.instance.user = self.request.user
        return super().form_valid(form)